# Description of the Notebook

This notebook was made to modify templates of runfiles for the NonLinLoc software. 

Firstly, for the grid files you can use the Matching_VelModel notebook to create 2 .txt files that will be needed to modify the template of the runfile. It is adding the lines for 2 commands: VGGRID and LAYER. Secondly, it modifies the command lines for the runfile made for location of events. All the parameters needed can be entered here and the modifications will be made on the runfile.

In [1]:
base_dir = '/Users/lpapin/Documents/phd/projects/sw4/nonlinloc/'

For now everything is in local but since NonLinLoc is on the cluster, let's try to keep a similar organization than the creator:

- NLLoc as the main folder
- NLLoc/model for the model grid
- NLLoc/time for the travel-time grid
- NLLoc/obs for the picks
- NLLoc/run for the runfiles
- NLLoc/loc for the locations

NB: #LP is added to each line modified thanks to this notebook to remember which lines were the original ones.

# 1. Vel2Grid

First we're modifying the line giving the grid description VGGRID for Vel2Grid. **CORRECTION:** For the SVI project, no need to use the usual commands to create the model grid. Instead, use of Loïc's notebooks to produce a SimulPS.txt file in /model. Still the cells below are correct for other projects.

In [2]:
# Read the original .run file
with open(base_dir+'ttgrid_runfile_template.in', 'r') as file:
    lines = file.readlines()

# Read the second line directly from VGGRID_command.txt
with open(base_dir+'VGGRID_command.txt', 'r') as file:
    lines_to_add = file.readlines()
    vggrid_command = lines_to_add[1].strip() 

# Find and replace the VGGRID line in the .run file
for i, line in enumerate(lines):
    if line.startswith('VGGRID'):
        lines[i] = vggrid_command + ' #LP' + '\n'
        break

# Write the modified content back to a new file (or overwrite the original)
with open(base_dir+'ttgrid_runfile.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print("VGGRID line has been modified successfully.")

VGGRID line has been modified successfully.


Now we add the lines giving the velocity model layers.

In [3]:
# Read the second line directly from LAYER_command.txt
with open(base_dir+'LAYER_command.txt', 'r') as file:
    layer_lines = [line.strip() for line in file.readlines()[2:]]
layer_lines = [f"{line} #LP" for line in layer_lines]

# Find the line to insert the LAYER commands in the .run file
for i, line in enumerate(lines):
    if line.startswith('#LAYER depth Vp_top Vp_grad Vs_top Vs_grad p_top p_grad'):
        insertion_point = i + 1  # Insert after this line
        break

# Insert the LAYER lines at the specified point
modified_lines = lines[:insertion_point] + layer_lines + lines[insertion_point:]

# Write the modified content back to a new file (or overwrite the original)
with open(base_dir+'ttgrid_runfile.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in modified_lines)

print("LAYER lines have been added successfully.")

LAYER lines have been added successfully.


In [4]:
# What does it look like after modifying the .run file for the commands
with open(base_dir+'ttgrid_runfile.in', 'r') as file:
    lines = file.readlines()
    non_comment_lines = [line.strip() for line in lines if not line.strip().startswith('#') and line.strip()]
    for line in non_comment_lines:
        print(line)

VGOUT  ./model/layer
VGTYPE P
VGTYPE S
VGGRID 30 26 32 -124.5 48.0 0.0 0.37 0.55 3.0 VELOCITY #LP
LAYER 0.0 4.78 0 2.77 0 0 0 #LP
LAYER 3.0 5.23 0 3.03 0 0 0 #LP
LAYER 6.0 5.75 0 3.33 0 0 0 #LP
LAYER 9.0 6.34 0 3.68 0 0 0 #LP
LAYER 12.0 6.56 0 3.81 0 0 0 #LP
LAYER 15.0 6.68 0 3.90 0 0 0 #LP
LAYER 18.0 6.71 0 3.92 0 0 0 #LP
LAYER 21.0 6.80 0 3.96 0 0 0 #LP
LAYER 24.0 6.81 0 3.96 0 0 0 #LP
LAYER 27.0 6.92 0 3.99 0 0 0 #LP
LAYER 30.0 6.94 0 4.01 0 0 0 #LP
LAYER 33.0 7.03 0 4.06 0 0 0 #LP
LAYER 36.0 7.15 0 4.11 0 0 0 #LP
LAYER 39.0 7.29 0 4.18 0 0 0 #LP
LAYER 42.0 7.43 0 4.26 0 0 0 #LP
LAYER 45.0 7.55 0 4.34 0 0 0 #LP
LAYER 48.0 7.62 0 4.42 0 0 0 #LP
LAYER 51.0 7.72 0 4.48 0 0 0 #LP
LAYER 54.0 7.81 0 4.53 0 0 0 #LP
LAYER 57.0 7.87 0 4.56 0 0 0 #LP
LAYER 60.0 7.92 0 4.58 0 0 0 #LP
LAYER 63.0 7.95 0 4.60 0 0 0 #LP
LAYER 66.0 7.97 0 4.61 0 0 0 #LP
LAYER 69.0 8.00 0 4.62 0 0 0 #LP
LAYER 72.0 8.02 0 4.64 0 0 0 #LP
LAYER 75.0 8.05 0 4.65 0 0 0 #LP
LAYER 78.0 8.07 0 4.67 0 0 0 #LP
LAYER 81.0 8.10

**Now the input file to use Vel2Grid to create the model grid is ready.**

*Note on 9/9*: it needs a last minute change for the output root file name VGOUT before launching.

# 2. NLLoc

I need to define the commands LOCFILES, LOCGRID, and TRANS SIMPLE. Not changing for now the LOCSEARCH command but it may be needed.

In [5]:
# Determine the filenames for LOCFILES
input_filename_obs = '/projects/amt/shared/lpapin/NLLoc/SVI/obs/' # Picks
input_filename_ttgrid = '/projects/amt/shared/lpapin/NLLoc/SVI/time/layer' # Travel-time grid
output_filename_loc = '/projects/amt/shared/lpapin/NLLoc/SVI/loc/' # Location of the events

# Read the original .run file
with open(base_dir+'nlloc_runfile_template.in', 'r') as file:
    lines = file.readlines()

locfiles_command = 'LOCFILES ' + input_filename_obs + ' NLL_OBS ' + input_filename_ttgrid + ' ' + output_filename_loc
# Find and replace the LOCFILES line in the .run file
for i, line in enumerate(lines):
    if line.startswith('LOCFILES'):
        lines[i] = locfiles_command + ' #LP' + '\n'
        break

# Write the modified content back to a new file (or overwrite the original)
with open(base_dir+'nlloc_runfile.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print("LOCFILES line has been modified successfully.")

LOCFILES line has been modified successfully.


In [6]:
# Determine the command line for LOCGRID
locgrid_command = 'LOCGRID ' + vggrid_command[7:-8] + 'PROB_DENSITY SAVE'
for i, line in enumerate(lines):
    if line.startswith('LOCGRID') and line.strip().endswith('PROB_DENSITY  SAVE'):
        lines[i] = locgrid_command + ' #LP' + '\n'
        break

with open(base_dir+'nlloc_runfile.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)
    
print("LOCGRID line has been modified successfully.")

LOCGRID line has been modified successfully.


In [7]:
# Get the origins of the rectangle
parameters = vggrid_command.split()
latOrig = parameters[5]
longOrig = parameters[4]
rotAngle = 0.0

# Determine the command line for TRANS SIMPLE
transsimple_command = 'TRANS SIMPLE ' + f'{latOrig} ' + f'{longOrig} ' + f'{rotAngle}' 
for i, line in enumerate(lines):
    if line.startswith('TRANS SIMPLE '):
        lines[i] = transsimple_command + ' #LP' + '\n'
        break

with open(base_dir+'nlloc_runfile.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print("TRANS SIMPLE line has been modified successfully.")

TRANS SIMPLE line has been modified successfully.


In [8]:
# What does it look like after modifying the .run file for the commands
with open(base_dir+'nlloc_runfile.in', 'r') as file:
    lines = file.readlines()
    non_comment_lines = [line.strip() for line in lines if not line.strip().startswith('#') and line.strip()]
    for line in non_comment_lines:
        print(line)

CONTROL 1 12345
TRANS SIMPLE 48.0 -124.5 0.0 #LP
LOCSIG Lois Papin -- UO
LOCFILES /projects/amt/shared/lpapin/NLLoc/SVI/obs/ NLL_OBS /projects/amt/shared/lpapin/NLLoc/SVI/time/layer /projects/amt/shared/lpapin/NLLoc/SVI/loc/ #LP
LOCHYPOUT SAVE_NLLOC_ALL SAVE_HYPOINV_SUM
LOCSEARCH OCT 17 11 6 0.001 100000 10000 0 0
LOCMETH EDT_OT_WT 9999.0 4 -1 -1 -1 -1 -1 1
LOCGAU 0.2 0.0
LOCGAU2 0.01 0.05 2.0
LOCPHASEID  P   P
LOCPHASEID  S   S
LOCQUAL2ERR 0.1 0.5 1.0 2.0 99999.9
LOCGRID 30 26 32 -124.5 48.0 0.0 0.37 0.55 3.0 PROB_DENSITY SAVE #LP
LOCPHSTAT 9999.0 -1 9999.0 1.0 1.0 9999.9 -9999.9 9999.9
LOCANGLES ANGLES_YES 5
LOCMAG ML_HB 1.0 1.110 0.00189


# Left to do
Verify what's need to be modified for Grid2Time and change accordingly the filenames and commands.